In [54]:
#import modules
!pip install Fuzzywuzzy

import numpy as np
import pandas as pd
from difflib import SequenceMatcher
from fuzzywuzzy import fuzz
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity, pairwise_distances, manhattan_distances,euclidean_distances
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import csv
import io
import random


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
import requests
import csv

# Download the CSV file from GitHub
url = 'https://raw.githubusercontent.com/ykwong1223/Principle-DS-new/main/Netflix.csv'
response = requests.get(url)
content = response.content.decode('utf-8')
csvreader = csv.reader(content.splitlines())
raw = pd.read_csv(io.StringIO(content))


In [56]:
#Import Dataset

raw=pd.DataFrame(raw)

#Remove duplicate and check for null
print(raw.shape)
raw.drop_duplicates(keep='last', inplace=True)
missing=raw.isnull().sum()
print(missing)

(8806, 12)
show_id            0
type               0
title              0
director        2634
cast             824
country          831
date_added        10
release_year       0
rating             4
duration           0
listed_in          0
description        0
dtype: int64


In [57]:
#Replace null and check again


for i in ["country","date_added","rating"]:
  raw[i].fillna(raw[i].mode().iloc[0], inplace=True)

for i in ["director","cast"]:
  raw[i].fillna("", inplace=True)

missing=raw.isnull().sum()
print(missing)

show_id         0
type            0
title           0
director        0
cast            0
country         0
date_added      0
release_year    0
rating          0
duration        0
listed_in       0
description     0
dtype: int64


In [58]:
#remove spcae and lower case the words and combine
def data_clean(k):
  return str.lower(k.replace(" ",""))
raw["combine"]=raw["title"].apply(data_clean)+" "+raw["cast"].apply(data_clean)+" "+raw["director"].apply(data_clean)+" "+raw["description"].apply(data_clean)

In [59]:
#Using Sequence Matcher to find similarity of description and choose the 10 highest similarity contents

def seq_similar(nm):
    j = raw[raw['title'] == nm].index[0]
    string1 = str(raw.loc[j, "combine"])
    for i in range(len(raw)):
      string2 = str(raw.loc[i, "combine"])
      sequence_ratio=SequenceMatcher(None, string1, string2).ratio()
      raw.at[i, "seq_ratio"] = sequence_ratio
    subset=raw[["title","seq_ratio"]]
    subset=subset.sort_values(by='seq_ratio',ascending=False)
    return subset

seq_similar("Naruto")[1:11]

,title,seq_ratio
74,The World's Most Amazing Vacation Rentals,0.330827
1858,Fida,0.330233
3989,Stay Tuned!,0.321678
4474,Santo Cachón,0.318318
1416,Night Stalker: The Hunt for a Serial Killer,0.317308
5569,Samurai Gourmet,0.314770
4168,Examination of Conscience,0.312020
5510,President,0.309524
4911,The Rachel Divide,0.308094
2248,The Claudia Kishi Club,0.308081


In [60]:
#Using Fuzzywuzzy to find similarity of description and choose the 10 highest similarity contents

def fuz_ratio(nm):
    j = raw[raw['title'] == nm].index[0]
    string1 = str(raw.loc[j, "combine"])
    for i in range(len(raw)):
      string2 = str(raw.loc[i, "combine"])
      # Calculate similarity using the ratio method (Levenshtein distance)
      similarity_ratio = fuzz.ratio(string1, string2)
      raw.at[i, "fuz_ratio"] = similarity_ratio
    subset=raw[["title","fuz_ratio"]]
    subset=subset.sort_values(by='fuz_ratio',ascending=False)
    return subset

fuz_ratio("Naruto")[1:11]


,title,fuz_ratio
74,The World's Most Amazing Vacation Rentals,33.0
1858,Fida,33.0
4474,Santo Cachón,32.0
3989,Stay Tuned!,32.0
1416,Night Stalker: The Hunt for a Serial Killer,32.0
75,You vs. Wild: Out Cold,31.0
2248,The Claudia Kishi Club,31.0
4168,Examination of Conscience,31.0
5569,Samurai Gourmet,31.0
4911,The Rachel Divide,31.0


In [61]:
#Using Dice Coefficient to find similarity of description and choose the 10 highest similarity contents
def dice_coefficient(nm):
    j = raw[raw['title'] == nm].index[0]
    string1 = str(raw.loc[j, "combine"])
    for i in range(len(raw)):
      if i!=j:
        string2 = str(raw.loc[i, "combine"])
        string1=set(string1)
        string2=set(string2)
        intersection_size = len(string1.intersection(string2))
        dice_coefficient = (2 * intersection_size) / (len(string1) + len(string2))
      else:
        dice_coefficient=1.1
      raw.at[i, "dice_coefficient"] = dice_coefficient
    subset=raw[["title","dice_coefficient"]]
    subset=subset.sort_values(by='dice_coefficient',ascending=False)
    return subset


dice_coefficient("Naruto")[1:11]

,title,dice_coefficient
11,Bangkok Breaking,1.000000
7684,Our House,1.000000
4040,That Thing Called Tadhana,0.981132
7784,Power Rangers Wild Force,0.981132
4077,The Breakup Playlist,0.981132
4047,No Other Woman,0.981132
7827,Ram Teri Ganga Maili,0.981132
6518,Come and Find Me,0.981132
7841,Recall,0.981132
7844,Red Dawn,0.981132


In [62]:
#Using Jaccard Coefficient to find similarity of description and choose the 10 highest similarity contents
def jaccard_coefficient(nm):
    j = raw[raw['title'] == nm].index[0]
    string1 = str(raw.loc[j, "combine"])
    for i in range(len(raw)):
      if i!=j:
        string2 = str(raw.loc[i, "combine"])
        string1 = set(string1)
        string2 = set(string2)
        intersection_size = len(string1.intersection(string2))
        union_size = len(string1.union(string2))
        jaccard_coefficient = intersection_size / union_size
      else:
        jaccard_coefficient=1.1
      raw.at[i, "jaccard_coefficient"] = jaccard_coefficient
    subset=raw[["title","jaccard_coefficient"]]
    subset=subset.sort_values(by="jaccard_coefficient",ascending=False)
    return subset

jaccard_coefficient("Naruto")[1:11]

,title,jaccard_coefficient
11,Bangkok Breaking,1.000000
7684,Our House,1.000000
4040,That Thing Called Tadhana,0.962963
7784,Power Rangers Wild Force,0.962963
4077,The Breakup Playlist,0.962963
4047,No Other Woman,0.962963
7827,Ram Teri Ganga Maili,0.962963
6518,Come and Find Me,0.962963
7841,Recall,0.962963
7844,Red Dawn,0.962963


In [63]:
#Vectorize "combine" column to matrx and use cosine similarity to find similarity of description and choose the 10 highest similarity contents
count = CountVectorizer().fit_transform(raw['combine'])
cosine_similar_score = cosine_similarity(count, count)

def cosine_similar(name):
    indx = raw.index[raw['title'] == name].tolist()
    score=list(enumerate(cosine_similar_score[indx[0]]))
    raw["score"]=[m[1] for m in score]
    subset=raw[["title","score"]]
    subset=subset.sort_values(by='score',ascending=False)
    return subset

cosine_similar("Naruto")[1:11]

,title,score
63,Naruto the Movie: Ninja Clash in the Land of Snow,0.340207
57,Naruto Shippûden the Movie: Bonds,0.322749
56,Naruto Shippuden the Movie: Blood Prison,0.298142
7559,Naruto Shippuden : Blood Prison,0.298142
58,Naruto Shippûden the Movie: The Will of Fire,0.272166
59,Naruto Shippuden: The Movie,0.218218
62,Naruto the Movie 3: Guardians of the Crescent ...,0.216506
61,Naruto the Movie 2: Legend of the Stone of Gelel,0.149071
60,Naruto Shippuden: The Movie: The Lost Tower,0.125988
2658,Bleach,0.091287


In [64]:
# Random select 10 contents
def rnd_model(nm):
    subset=raw[["title"]]
    subset=subset.sample(frac=1)
    return subset


rnd_model("Naruto")[1:11]


,title
2100,Septembers of Shiraz
1849,Kartini: Princess of Java
801,Never Back Down 2: The Beatdown
7898,Rukh
7853,Remember
4882,Thi Mai
8082,Star Wars: Episode VIII: The Last Jedi
5106,Bibi & Tina: Tohuwabohu Total
942,Mystic River
3607,Dollar


In [65]:
#Build a data fram and use euclidean distance and manhattan distance to evaluate the models

performance_df = pd.DataFrame(columns=['Model', 'Total Euclidean Distance', 'Total Manhattan Distance'], index=range(6))
performance_df["Model"] = ["Sequence Matcher", "Fuzzy Ratio", "Dice Coefficient", "Jaccard Coefficient", "Cosine Similarity", "Random"]

vectorizer = TfidfVectorizer()
tfidf_vectors = vectorizer.fit_transform(raw["combine"])
total_euclidean_distance=0

recommendation_models=[seq_similar, fuz_ratio, dice_coefficient, jaccard_coefficient, cosine_similar, rnd_model]

for f in range(6):
  performance_df.loc[f, "Total Euclidean Distance"]=0
  performance_df.loc[f, "Total Manhattan Distance"]=0

for k in range(100):
  random_int = random.randint(0, len(raw)-1)
  name_selected=raw.loc[random_int,"title"]
  for f in range(6):
    mthd=recommendation_models[f]
    subset=mthd(name_selected)
    for g in range(1,11):
        j = raw[raw['title'] == subset.iloc[g]["title"]].index[0]
        manhattan_distance = manhattan_distances(tfidf_vectors[random_int], tfidf_vectors[j])
        euclidean_distance = euclidean_distances(tfidf_vectors[random_int], tfidf_vectors[j])
        performance_df.loc[f, "Total Euclidean Distance"]=euclidean_distance+performance_df.loc[f, "Total Euclidean Distance"]
        performance_df.loc[f, "Total Manhattan Distance"]=manhattan_distance+performance_df.loc[f, "Total Manhattan Distance"]

print(performance_df)

                 Model Total Euclidean Distance Total Manhattan Distance
0     Sequence Matcher     [1411.1921786512332]      [6101.429049661804]
1          Fuzzy Ratio     [1411.2686717782917]      [6105.661848753184]
2     Dice Coefficient     [1413.6302402063288]      [7035.729318615015]
3  Jaccard Coefficient     [1413.6302402063288]      [7035.729318615015]
4    Cosine Similarity     [1366.8291108764136]      [6524.151162046097]
5               Random     [1414.1288750344004]      [7028.296164073989]


In [66]:
# total_euclidean_distance3=0
# total_manhattan_distance3=0
# for k in range(100):
#   random_int = random.randint(0, len(raw)-1)
#   for g in range(1,11):
#     random_int2=random.randint(0, len(raw)-1)
#     manhattan_distance = manhattan_distances(tfidf_vectors[random_int], tfidf_vectors[random_int2])
#     euclidean_distance = euclidean_distances(tfidf_vectors[random_int], tfidf_vectors[random_int2])
#     total_euclidean_distance3=euclidean_distance+total_euclidean_distance3
#     total_manhattan_distance3=manhattan_distance+total_manhattan_distance3

# print(total_euclidean_distance3)
# print(total_manhattan_distance3)

In [67]:
# #Using Sequence Matcher to find similarity of description and choose the 10 highest similarity contents

# def seq_similar_2(nm):
#     j = raw[raw['title'] == nm].index[0]
#     string1 = str(raw.loc[j, "combine_2"])
#     for i in range(len(raw)):
#       string2 = str(raw.loc[i, "combine_2"])
#       sequence_ratio=SequenceMatcher(None, string1, string2).ratio()
#       raw.at[i, "seq_ratio_2"] = sequence_ratio
#     subset=raw[["title","seq_ratio_2"]]
#     subset=subset.sort_values(by='seq_ratio_2',ascending=False)
#     return subset

# seq_similar_2("Naruto Shippuden the Movie: Blood Prison")[0:10]

In [68]:
# #Using Fuzzywuzzy to find similarity of description and choose the 10 highest similarity contents

# def fuz_ratio_2(nm):
#     j = raw[raw['title'] == nm].index[0]
#     string1 = str(raw.loc[j, "combine_2"])
#     for i in range(len(raw)):
#       string2 = str(raw.loc[i, "combine_2"])
#       # Calculate similarity using the ratio method (Levenshtein distance)
#       similarity_ratio = fuzz.ratio(string1, string2)
#       raw.at[i, "fuz_ratio_2"] = similarity_ratio
#     subset=raw[["title","fuz_ratio_2"]]
#     subset=subset.sort_values(by='fuz_ratio_2',ascending=False)
#     return subset

# fuz_ratio_2("Naruto Shippuden the Movie: Blood Prison")[0:10]

In [69]:
# def dice_coefficient_2(nm):
#     j = raw[raw['title'] == nm].index[0]
#     string1 = str(raw.loc[j, "combine_2"])
#     for i in range(len(raw)):
#       string2 = str(raw.loc[i, "combine_2"])
#       string1=set(string1)
#       string2=set(string2)
#       intersection_size = len(string1.intersection(string2))
#       dice_coefficient = (2 * intersection_size) / (len(string1) + len(string2))
#       raw.at[i, "dice_coefficient_2"] = dice_coefficient
#     subset=raw[["title","dice_coefficient_2"]]
#     subset=subset.sort_values(by='dice_coefficient_2',ascending=False)
#     return subset


# dice_coefficient_2("Welcome")[0:10]

In [70]:
# def jaccard_similar_2(nm):
#     j = raw[raw['title'] == nm].index[0]
#     string1 = str(raw.loc[j, "combine_2"])
#     for i in range(len(raw)):
#       if i!=j:
#         string2 = str(raw.loc[i, "combine_2"])
#         string1 = set(string1)
#         string2 = set(string2)
#         intersection_size = len(string1.intersection(string2))
#         union_size = len(string1.union(string2))
#         jaccard_similarity = intersection_size / union_size
#         raw.at[i, "jaccard_similarity_2"] = jaccard_similarity
#     subset=raw[["title","jaccard_similarity_2"]]
#     subset=subset.sort_values(by='jaccard_similarity_2',ascending=False)
#     return subset

# jaccard_similar_2("Pokémon Master Journeys: The Series")[0:10]

In [71]:
# count = CountVectorizer().fit_transform(raw['combine'])
# cosine_similar_score = cosine_similarity(count, count)

# def cosine_similar_2(name):
#     indx = raw.index[raw['title'] == name].tolist()
#     print(indx)
#     score=list(enumerate(cosine_similar_score[indx[0]]))
#     raw["score_2"]=[m[1] for m in score]
#     subset=raw[["title","score_2"]]
#     subset=subset.sort_values(by='score_2',ascending=False)
#     return subset

# cosine_similar_2("Pokémon Master Journeys: The Series")[0:10]

In [72]:
# performance_df_2 = pd.DataFrame(columns=['Model', 'Total Euclidean Distance',"Total Manhattan Distance"], index=range(5))
# performance_df_2["Model"] = ["Sequence Similiar 2", "Fuzzy Ratio 2", "Dice Coefficient 2", "Jaccard Similarity 2", "Cosine Similarity 2"]

# vectorizer = TfidfVectorizer()
# tfidf_vectors = vectorizer.fit_transform(raw["combine_2"])

# recommendation_models=[seq_similar_2, fuz_ratio_2, dice_coefficient_2, jaccard_similar_2, cosine_similar_2]
# for f in range(5):
#   total_euclidean_distance=0
#   total_manhattan_distance=0
#   mthd=recommendation_models[f]
#   for k in range(100):
#     random_int = random.randint(0, len(raw)-1)
#     name_selected=raw.loc[random_int,"title"]
#     subset=mthd(name_selected)
#     for g in range(1,11):
#       j = raw[raw['title'] == subset.iloc[g]["title"]].index[0]
#       manhattan_distance = manhattan_distances(tfidf_vectors[random_int], tfidf_vectors[j])
#       euclidean_distance = euclidean_distances(tfidf_vectors[random_int], tfidf_vectors[j])
#       total_euclidean_distance=euclidean_distance+total_euclidean_distance
#       total_manhattan_distance=manhattan_distance+total_manhattan_distance
#   performance_df_2.loc[f, "Total Euclidean Distance"]=total_euclidean_distance
#   performance_df_2.loc[f, "Total Manhattan Distance"]=total_manhattan_distance

# print(performance_df_2)

In [73]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error, r2_score
# # from sklearn.preprocessing import LabelEncoder

# # Load the Netflix movie dataset
# raw = pd.read_csv("Netflix.csv")

# # Handling missing values
# for i in ["country", "date_added", "rating"]:
#     raw[i].fillna(raw[i].mode().iloc[0], inplace=True)

# for i in ["director", "cast"]:
#     raw[i].fillna("", inplace=True)

# # Preprocess 'duration' column
# raw['duration'] = raw['duration'].str.extract('(\d+)')
# raw['duration'] = raw['duration'].astype(float)

# # Select relevant features for prediction (example: duration and release_year)
# features = raw[['duration', 'release_year']]
# target = raw['rating']

# # Handling missing values in features
# features.fillna(features.mean(), inplace=True)

# # Label encode the target variable
# label_encoder = LabelEncoder()
# target_encoded = label_encoder.fit_transform(target)

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(features, target_encoded, test_size=0.2, random_state=42)

# # Create a linear regression model
# model = LinearRegression()

# # Fit the model to the training data
# model.fit(X_train, y_train)

# # Predict ratings for the test set
# y_pred = model.predict(X_test)

# # Model evaluation
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print("Mean Squared Error:", mse)
# print("R-squared:", r2)


In [74]:
#performance_df = pd.DataFrame(columns=['Model', 'cs simil'], index=range(5))
# performance_df["Model"] = ["Sequence Similiar", "Fuzzy Ratio", "Dice Coefficient", "Jaccard Similarity", "Euclidea Distance"]

# vectorizer = TfidfVectorizer()
# tfidf_vectors = vectorizer.fit_transform(raw["combine"])

# recommendation_models=[seq_similar, fuz_ratio, dice_coefficient, jaccard_similar, euclidean_distance]
# for f in range(5):
#   total_cs_simil=0
#   mthd=recommendation_models[f]
#   for k in range(100):
#     random_int = random.randint(0, len(raw)-1)
#     name_selected=raw.loc[random_int,"title"]
#     subset=mthd(name_selected)
#     for g in range(1,11):
#       j = raw[raw['title'] == subset.iloc[g]["title"]].index[0]
#       cs_simil = cosine_similarity(tfidf_vectors[random_int], tfidf_vectors[j])
#       total_cs_simil=cs_simil+total_cs_simil

#   performance_df.loc[f, "cs simil"]=total_cs_simil


In [75]:
# count = CountVectorizer().fit_transform(raw['combine'])
# euclidean_distance_score = euclidean_distances(count, count)

# def euclidean_distance(name):
#     indx = raw.index[raw['title'] == name].tolist()
#     print(indx)
#     score=list(enumerate(euclidean_distance_score[indx[0]]))
#     raw["score"]=[m[1] for m in score]
#     subset=raw[["title","score"]]
#     subset=subset.sort_values(by='score',ascending=True)
#     return subset

# euclidean_distance("Pokémon Master Journeys: The Series")[1:10]